In [ ]:
SEED = 123
NUM_BITS, NUM_CHECKS = 10, 6
CHECK_DEGREE, BIT_DEGREE = 5, 3
if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
    raise ValueError("The Tanner graph of the code must be bipartite.")
code = qec.random_regular_code(
    NUM_BITS,
    NUM_CHECKS,
    BIT_DEGREE,
    CHECK_DEGREE,
    qec.Rng(SEED)
    )
hgpc = qec.hypergraph_product(code, code)
num_sites = 2 * hgpc.length() + hgpc.num_x_logicals() + hgpc.num_z_logicals()
num_logicals = hgpc.num_x_logicals() + hgpc.num_z_logicals()
error = "0" * num_sites
string_state = "+" * num_logicals + error
error_state = create_custom_product_state(string=string_state, form="Right-canonical")
constraints_tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
logicals_tensors = [COPY_LEFT, XOR_BULK, SWAP, XOR_RIGHT]\

CHI_MAX = 64

In [ ]:
constraints_sites = css_code_constraint_sites(hgpc)
logicals_sites = css_code_logicals_sites(hgpc)

In [ ]:
renormalise = True
error_state = apply_bias_channel(
    basis_mps=error_state, basis_string=string_state, prob_channel=0.1
)

In [ ]:
error_state = apply_constraints(
    error_state,
    constraints_sites[0],
    constraints_tensors,
    renormalise=renormalise,
    chi_max=CHI_MAX,
)
error_state = apply_constraints(
    error_state,
    constraints_sites[1],
    constraints_tensors,
    renormalise=renormalise,
    chi_max=CHI_MAX,
)
error_state = apply_constraints(
    error_state,
    logicals_sites,
    logicals_tensors,
    renormalise=renormalise,
    chi_max=CHI_MAX,
)

In [ ]:
sites_to_marginalise = list(range(num_logicals, len(error_state)))

In [ ]:
logicals = marginalise(mps=error_state, sites_to_marginalise=sites_to_marginalise)

In [ ]:
num_sites_dmrg = len(logical)
mps_start_dmrg = create_simple_product_state(num_sites_dmrg, which="+")
engine = DephasingDMRG(
    mps_dmrg_start,
    message, #??? all zeros
    chi_max=chi_max_dmrg,
    cut=cut,
    mode="LA",
    silent=silent,
)